<h1 align="left">
  <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://autonomousvision.github.io/py123d/_static/123D_logo_transparent_white.svg" width="500">
    <source media="(prefers-color-scheme: light)" srcset="https://autonomousvision.github.io/py123d/_static/123D_logo_transparent_black.svg" width="500">
    <img alt="Logo" src="https://autonomousvision.github.io/py123d/_static/123D_logo_transparent_black.svg" width="500">
  </picture>
  <h2 align="left">123D: Geometry Tutorial</h1>
</h1>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from py123d.geometry import (
    BoundingBoxSE2,
    BoundingBoxSE3,
    EulerAngles,
    Point2D,
    Point3D,
    Polyline2D,
    Polyline3D,
    PolylineSE2,
    PoseSE2,
    PoseSE3,
    Quaternion,
    Vector2D,
    Vector3D,
)

## 4.1 Introduction

The `py123d.geometry` module provides geometric primitives for working with autonomous driving data. All classes share a common design:

- **Array-backed**: Each object wraps a NumPy array, accessible via the `.array` property.
- **Immutable**: Objects cannot be modified after creation.
- **Index enums**: Named indices (e.g., `Point3DIndex.X`) provide readable array access.
- **Interoperable**: Objects can be constructed from arrays (`from_array`) and converted to NumPy arrays (`np.array(obj)`).

This tutorial requires **no dataset downloads** -- all examples are self-contained.

## 4.2 Points and Vectors

Points represent locations in space, while vectors represent displacements. The key difference is how they interact:
- `Point - Point = Vector` (displacement between two locations)
- `Point + Vector = Point` (translate a location)
- `Vector + Vector = Vector` (combine displacements)

### 4.2.1 `Point2D` and `Point3D`

In [ ]:
# Create points directly
p2d = Point2D(1.0, 2.0)
print("Point2D:\t", p2d)
print("x, y:\t\t", p2d.x, p2d.y)
print("Array:\t\t", p2d.array)

p3d = Point3D(1.0, 2.0, 3.0)
print("\nPoint3D:\t", p3d)
print("x, y, z:\t", p3d.x, p3d.y, p3d.z)

# Create from arrays
p2d_from_array = Point2D.from_array(np.array([5.0, 6.0]))
print("\nFrom array:\t", p2d_from_array)

# 3D to 2D projection
print("3D -> 2D:\t", p3d.point_2d)

### 4.2.2 `Vector2D` and `Vector3D`

In [ ]:
v1 = Vector2D(3.0, 4.0)
v2 = Vector2D(1.0, 0.0)

print("v1:\t\t", v1)
print("v1.magnitude:\t", v1.magnitude)

# Arithmetic operations
print("\nv1 + v2:\t", v1 + v2)
print("v1 - v2:\t", v1 - v2)
print("v1 * 2:\t\t", v1 * 2)
print("v1 / 2:\t\t", v1 / 2)
print("-v1:\t\t", -v1)

### 4.2.3 Point-Vector Arithmetic

In [ ]:
p = Point2D(1.0, 1.0)
v = Vector2D(2.0, 3.0)

# Translate a point by a vector
p_translated = p + v
print("Point + Vector:\t", p_translated, f"(type: {type(p_translated).__name__})")

# Displacement between two points
p2 = Point2D(4.0, 5.0)
displacement = p2 - p
print("Point - Point:\t", displacement, f"(type: {type(displacement).__name__})")

# Move a point backward
p_back = p - v
print("Point - Vector:\t", p_back, f"(type: {type(p_back).__name__})")

In [ ]:
# Visualize point-vector arithmetic
fig, ax = plt.subplots(figsize=(6, 6))

# Points
ax.plot(p.x, p.y, "ko", markersize=8, label="p = (1, 1)")
ax.plot(p_translated.x, p_translated.y, "bs", markersize=8, label=f"p + v = ({p_translated.x}, {p_translated.y})")
ax.plot(p2.x, p2.y, "r^", markersize=8, label=f"p2 = ({p2.x}, {p2.y})")

# Vectors as arrows
ax.annotate(
    "", xy=(p_translated.x, p_translated.y), xytext=(p.x, p.y), arrowprops=dict(arrowstyle="->", color="blue", lw=2)
)
ax.text(2.0, 2.8, f"v = ({v.x}, {v.y})", color="blue", fontsize=10)

ax.annotate("", xy=(p2.x, p2.y), xytext=(p.x, p.y), arrowprops=dict(arrowstyle="->", color="red", lw=2))
ax.text(2.8, 2.5, f"p2 - p = ({displacement.x}, {displacement.y})", color="red", fontsize=10)

ax.set_xlim(-0.5, 6)
ax.set_ylim(-0.5, 6)
ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_title("Point-Vector Arithmetic")
plt.show()

## 4.3 Rotations
Rotations in 123D have three different representations, that you might encounter:

| Representation | Parameters | \#Params | Notes |
|---|---|---|---|
| **Euler angles** | roll, pitch, yaw | 3 | Intuitive but subject to gimbal lock. Rotation order is intrinsic Z-Y'-X'' (yaw-pitch-roll). |
| **Quaternions** | qw, qx, qy, qz | 4 | Compact, no gimbal lock, used internally with scale-first convention in `PoseSE3`. |
| **Rotation matrices** | 3×3 matrix | 9 | Useful for matrix operations. 9 parameters but only 3 DOF due to orthonormality constraints. |

All three representations are mathematically equivalent, but **quaternions are the default representation in 123D**. 
Please be cautious with Euler angles due to gimbal lock and rotation-order conventions. 

Note that `PosseSE2` internally simply uses yaw angles in radian, but provides a 2×2 rotation matrix.


### 4.3.1 `EulerAngles`

In [ ]:
# Create Euler angles: 90-degree yaw rotation (turn left)
euler = EulerAngles(roll=0.0, pitch=0.0, yaw=np.pi / 2)
print("EulerAngles:\t", euler)
print("roll:\t\t", euler.roll)
print("pitch:\t\t", euler.pitch)
print("yaw:\t\t", euler.yaw)

# Convert to rotation matrix
print("\nRotation matrix:")
print(euler.rotation_matrix.round(4))

# Convert to quaternion
print("\nQuaternion:\t", euler.quaternion)

### 4.3.2 `Quaternion`

In [ ]:
# Identity quaternion (no rotation)
q_identity = Quaternion(qw=1.0, qx=0.0, qy=0.0, qz=0.0)
print("Identity quaternion:\t", q_identity)
print("Rotation matrix:")
print(q_identity.rotation_matrix)

# Create from rotation matrix
R_90 = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]], dtype=np.float64)  # 90-degree yaw
q_from_R = Quaternion.from_rotation_matrix(R_90)
print("\nFrom rotation matrix:", q_from_R)
print("Euler angles:\t\t", q_from_R.euler_angles)

### 4.3.3 Conversion Roundtrip

All rotation representations are consistent -- converting between them preserves the rotation:

In [ ]:
# Start with Euler angles
euler_original = EulerAngles(roll=0.1, pitch=0.2, yaw=0.3)

# Euler -> Quaternion -> Rotation Matrix -> Euler
quat = euler_original.quaternion
rot_matrix = quat.rotation_matrix
euler_roundtrip = EulerAngles.from_rotation_matrix(rot_matrix)

print("Original:\t", euler_original)
print("Roundtrip:\t", euler_roundtrip)
print("Match:\t\t", np.allclose(euler_original.array, euler_roundtrip.array))

## 4.4 Poses

Poses combine translation and rotation into a single rigid-body transformation:
- `PoseSE2`: 2D pose with (x, y, yaw)
- `PoseSE3`: 3D pose with (x, y, z, qw, qx, qy, qz)

### 4.4.1 `PoseSE2`

In [ ]:
# Create a 2D pose
pose_2d = PoseSE2(x=5.0, y=3.0, yaw=np.pi / 4)  # 45-degree rotation
print("PoseSE2:\t", pose_2d)
print("Point:\t\t", pose_2d.point_2d)
print("Yaw (deg):\t", np.degrees(pose_2d.yaw))

# 2x2 rotation matrix
print("\nRotation matrix:")
print(pose_2d.rotation_matrix.round(4))

# 3x3 transformation matrix
print("\nTransformation matrix:")
print(pose_2d.transformation_matrix.round(4))

# Identity pose
print("\nIdentity:\t", PoseSE2.identity())

### 4.4.2 `PoseSE3`

In [ ]:
# Create a 3D pose with identity rotation
pose_3d = PoseSE3(x=10.0, y=20.0, z=1.0, qw=1.0, qx=0.0, qy=0.0, qz=0.0)
print("PoseSE3:\t", pose_3d)
print("Point3D:\t", pose_3d.point_3d)
print("Point2D:\t", pose_3d.point_2d)
print("Quaternion:\t", pose_3d.quaternion)
print("Euler angles:\t", pose_3d.euler_angles)

# Create from rotation + translation
euler = EulerAngles(roll=0.0, pitch=0.0, yaw=np.pi / 2)
translation = Point3D(5.0, 10.0, 0.0)
pose_from_R_t = PoseSE3.from_R_t(rotation=euler, translation=translation)
print("\nFrom R, t:\t", pose_from_R_t)
print("Yaw (deg):\t", np.degrees(pose_from_R_t.yaw))

# SE3 -> SE2 projection
print("\nSE3 -> SE2:\t", pose_from_R_t.pose_se2)

### 4.4.3 Construction Methods

`PoseSE3` supports flexible construction from various rotation and translation types:

In [ ]:
# From rotation matrix + numpy array
R = np.eye(3, dtype=np.float64)
t = np.array([1.0, 2.0, 3.0])
p1 = PoseSE3.from_R_t(rotation=R, translation=t)

# From Quaternion + Vector3D
q = Quaternion(1.0, 0.0, 0.0, 0.0)
v = Vector3D(1.0, 2.0, 3.0)
p2 = PoseSE3.from_R_t(rotation=q, translation=v)

# From 4x4 transformation matrix
T = np.eye(4, dtype=np.float64)
T[:3, 3] = [1.0, 2.0, 3.0]
p3 = PoseSE3.from_transformation_matrix(T)

print("From R, t (arrays):\t", p1)
print("From Q, V (typed):\t", p2)
print("From 4x4 matrix:\t", p3)
print("All equal:\t\t", p1 == p2 == p3)

### 4.4.4 Transformation Matrix Roundtrip

In [ ]:
pose = PoseSE3.from_R_t(
    rotation=EulerAngles(roll=0.1, pitch=0.2, yaw=0.3),
    translation=Point3D(10.0, 20.0, 5.0),
)

# Pose -> Transformation matrix -> Pose
T = pose.transformation_matrix
pose_roundtrip = PoseSE3.from_transformation_matrix(T)

print("Original:\t", pose)
print("Roundtrip:\t", pose_roundtrip)
print("Match:\t\t", np.allclose(pose.array, pose_roundtrip.array))

print("\n4x4 Transformation matrix:")
print(T.round(4))

## 4.5 Index Enums

All geometry classes use `IntEnum`-based index enums for readable array access. This makes code self-documenting when working with raw arrays.

In [ ]:
from py123d.geometry import (
    Corners3DIndex,
    MatrixSE3Index,
    PoseSE3Index,
)

# Individual element access
pose_array = pose.array
print("x:\t", pose_array[PoseSE3Index.X])
print("y:\t", pose_array[PoseSE3Index.Y])
print("z:\t", pose_array[PoseSE3Index.Z])
print("qw:\t", pose_array[PoseSE3Index.QW])

# Slice access (via classproperty)
print("\nXYZ:\t", pose_array[PoseSE3Index.XYZ])
print("QUAT:\t", pose_array[PoseSE3Index.QUATERNION])

# Matrix indexing
T = pose.transformation_matrix
print("\nRotation block (3x3):")
print(T[MatrixSE3Index.ROTATION].round(4))
print("\nTranslation (3,):\t", T[MatrixSE3Index.TRANSLATION])

## 4.6 Bounding Boxes

Bounding boxes are defined by a center pose and extents (length, width, and optionally height). They are used for object detection annotations.

### 4.6.1 `BoundingBoxSE2`

In [ ]:
# Create a rotated 2D bounding box
center = PoseSE2(x=5.0, y=3.0, yaw=np.pi / 6)  # 30-degree rotation
bbox_2d = BoundingBoxSE2(center_se2=center, length=4.0, width=2.0)

print("BoundingBoxSE2:\t", bbox_2d)
print("Center:\t\t", bbox_2d.center_se2)
print("Length:\t\t", bbox_2d.length)
print("Width:\t\t", bbox_2d.width)
print("Area:\t\t", bbox_2d.shapely_polygon.area)

# Get corners
corners = bbox_2d.corners_array
print("\nCorners (4x2):")
print(corners.round(4))

# Named corner access
corners_dict = bbox_2d.corners_dict
for name, corner in corners_dict.items():
    print(f"  {name.name}:\t{corner}")

In [ ]:
# Visualize multiple bounding boxes at different orientations
fig, ax = plt.subplots(figsize=(8, 8))

cmap = plt.get_cmap("tab10")
yaw_degrees = np.arange(0, 180, 30)
colors = cmap(np.linspace(0, 1, len(yaw_degrees)))
x_offsets = np.linspace(-5, 5, len(yaw_degrees))
for i, (yaw_deg, x_offset) in enumerate(zip(yaw_degrees, x_offsets)):
    yaw_rad = np.radians(yaw_deg)
    bbox = BoundingBoxSE2(
        center_se2=PoseSE2(x=x_offset, y=0.0, yaw=yaw_rad),
        length=4.0,
        width=2.0,
    )
    corners = bbox.corners_array
    closed = np.vstack([corners, corners[0]])
    ax.fill(closed[:, 0], closed[:, 1], alpha=0.3, color=colors[i])
    ax.plot(closed[:, 0], closed[:, 1], color=colors[i], label=f"yaw={yaw_deg}°")

    dx = 2.0 * np.cos(yaw_rad)
    dy = 2.0 * np.sin(yaw_rad)
    ax.annotate(
        "",
        xy=(x_offset + dx, dy),
        xytext=(x_offset, 0),
        arrowprops=dict(arrowstyle="->", color=colors[i], lw=1.5),
    )

ax.set_xlim(-8, 8)
ax.set_ylim(-4, 4)
ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
ax.legend(loc="upper right")
ax.set_title("BoundingBoxSE2 at different yaw angles")
plt.show()

### 4.6.2 `BoundingBoxSE3`

In [ ]:
# Create a 3D bounding box
center_3d = PoseSE3(x=10.0, y=5.0, z=1.0, qw=1.0, qx=0.0, qy=0.0, qz=0.0)
bbox_3d = BoundingBoxSE3(center_se3=center_3d, length=4.5, width=2.0, height=1.8)

print("BoundingBoxSE3:\t", bbox_3d)
print("Length:\t\t", bbox_3d.length)
print("Width:\t\t", bbox_3d.width)
print("Height:\t\t", bbox_3d.height)

# 8 corners of the 3D box
corners_3d = bbox_3d.corners_array
print("\nCorners (8x3):")
print(corners_3d.round(4))

# Named corner access
corners_3d_dict = bbox_3d.corners_dict
for name, corner in corners_3d_dict.items():
    print(f"  {name.name}:\t{corner}")

# SE3 -> SE2 projection
print("\nSE2 projection:\t", bbox_3d.bounding_box_se2)

In [ ]:
# Visualize the 3D bounding box
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection="3d")

# Get the 8 corners of the box
corners = bbox_3d.corners_array

# Define the 12 edges of the box by connecting corner indices
edges = [
    (Corners3DIndex.FRONT_LEFT_BOTTOM, Corners3DIndex.FRONT_RIGHT_BOTTOM),
    (Corners3DIndex.FRONT_RIGHT_BOTTOM, Corners3DIndex.BACK_RIGHT_BOTTOM),
    (Corners3DIndex.BACK_RIGHT_BOTTOM, Corners3DIndex.BACK_LEFT_BOTTOM),
    (Corners3DIndex.BACK_LEFT_BOTTOM, Corners3DIndex.FRONT_LEFT_BOTTOM),
    (Corners3DIndex.FRONT_LEFT_TOP, Corners3DIndex.FRONT_RIGHT_TOP),
    (Corners3DIndex.FRONT_RIGHT_TOP, Corners3DIndex.BACK_RIGHT_TOP),
    (Corners3DIndex.BACK_RIGHT_TOP, Corners3DIndex.BACK_LEFT_TOP),
    (Corners3DIndex.BACK_LEFT_TOP, Corners3DIndex.FRONT_LEFT_TOP),
    (Corners3DIndex.FRONT_LEFT_BOTTOM, Corners3DIndex.FRONT_LEFT_TOP),
    (Corners3DIndex.FRONT_RIGHT_BOTTOM, Corners3DIndex.FRONT_RIGHT_TOP),
    (Corners3DIndex.BACK_RIGHT_BOTTOM, Corners3DIndex.BACK_RIGHT_TOP),
    (Corners3DIndex.BACK_LEFT_BOTTOM, Corners3DIndex.BACK_LEFT_TOP),
]

# Plot the edges
for start, end in edges:
    ax.plot(corners[[start, end], 0], corners[[start, end], 1], corners[[start, end], 2], "b-")

# Plot the corners and their labels
for corner_idx in Corners3DIndex:
    x, y, z = corners[corner_idx]
    ax.scatter(x, y, z, c="r", marker="o")
    ax.text(x, y, z, f"  {corner_idx.name}", color="purple", fontsize=8)


# Plot the center
center = bbox_3d.center_se3
ax.scatter(center.x, center.y, center.z, c="g", marker="^", s=100)
ax.text(center.x, center.y, center.z, "  Center", color="green", fontsize=8)

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("BoundingBoxSE3 Visualization")

# Set aspect ratio to be equal
ax.set_box_aspect([np.ptp(corners[:, i]) for i in range(3)])

plt.show()

## 4.7 Polylines

Polylines represent sequences of connected points. They support interpolation, projection, and length computation. Three variants are available:
- `Polyline2D`: 2D path (x, y)
- `Polyline3D`: 3D path (x, y, z)
- `PolylineSE2`: 2D path with heading (x, y, yaw)

### 4.7.1 `Polyline2D`

In [ ]:
# Create a polyline from a numpy array
points = np.array(
    [
        [0.0, 0.0],
        [2.0, 1.0],
        [4.0, 0.5],
        [6.0, 2.0],
        [8.0, 1.0],
    ]
)
polyline = Polyline2D.from_array(points)

print("Length:\t\t", round(polyline.length, 4))
print("Array shape:\t", polyline.array.shape)

# Interpolate at a specific distance
mid_point = polyline.interpolate(polyline.length / 2)
print("\nMidpoint:\t", mid_point)

# Interpolate with normalized distance (0 to 1)
quarter_point = polyline.interpolate(0.25, normalized=True)
print("Quarter point:\t", quarter_point)

# Project a point onto the polyline
query_point = Point2D(4.0, 2.0)
distance_along = polyline.project(query_point)
print(f"\nProjection of {query_point}:\t{round(float(distance_along), 4)} meters along polyline")

In [ ]:
# Visualize polyline interpolation
fig, ax = plt.subplots(figsize=(10, 5))

# Draw the polyline
ax.plot(polyline.array[:, 0], polyline.array[:, 1], "b-o", linewidth=2, label="Polyline")

# Mark interpolated points along the polyline
n_samples = 20
distances = np.linspace(0.01, polyline.length - 0.01, n_samples)
interp_points = polyline.interpolate(distances)
ax.scatter(interp_points[:, 0], interp_points[:, 1], c=distances, cmap="viridis", s=60, zorder=5, label="Interpolated")
plt.colorbar(ax.collections[-1], ax=ax, label="Distance along polyline [m]")

# Mark the projected point
closest = polyline.interpolate(float(distance_along))
ax.plot(query_point.x, query_point.y, "r*", markersize=15, label="Query point")
ax.plot(closest.x, closest.y, "g*", markersize=15, label="Closest on polyline")
ax.plot([query_point.x, closest.x], [query_point.y, closest.y], "r--", alpha=0.5)

ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_title("Polyline2D: Interpolation and Projection")
plt.show()

### 4.7.2 `PolylineSE2`

`PolylineSE2` extends `Polyline2D` with heading information. It can be created from SE2 states or from a `Polyline2D` (where headings are inferred from the path direction).

In [ ]:
# Create from SE2 array (x, y, yaw)
# Create a smooth, curvy path using a sine wave
num_points = 30
x = np.linspace(0, 4 * np.pi, num_points)
y = 2 * np.sin(x / 2)

# Calculate yaw as the angle of the path's tangent
yaw = np.arctan2(np.gradient(y), np.gradient(x))

# Create the SE2 states array (x, y, yaw)
se2_states = np.stack([x, y, yaw], axis=-1)

polyline_se2 = PolylineSE2.from_array(se2_states)

# or from 2d points:
polyline_se2 = PolylineSE2.from_array(se2_states[..., PoseSE3Index.XY])


print("Length:\t\t", round(polyline_se2.length, 4))

# Interpolation returns PoseSE2 (with heading)
interp_pose = polyline_se2.interpolate(polyline_se2.length / 2)
print("Midpoint pose:\t", interp_pose)
print("Midpoint yaw:\t", round(np.degrees(interp_pose.yaw), 2), "degrees")

In [ ]:
# Visualize PolylineSE2 with heading arrows
fig, ax = plt.subplots(figsize=(8, 8))

# Draw the path
ax.plot(se2_states[:, 0], se2_states[:, 1], "b-o", linewidth=2, label="Path")

# Draw heading arrows at each waypoint
arrow_len = 0.5
for state in se2_states:
    x, y, yaw = state
    dx = arrow_len * np.cos(yaw)
    dy = arrow_len * np.sin(yaw)
    ax.annotate("", xy=(x + dx, y + dy), xytext=(x, y), arrowprops=dict(arrowstyle="->", color="red", lw=2))

# Sample interpolated poses along the path
n_samples = 15
distances = np.linspace(0.01, polyline_se2.length - 0.01, n_samples)
for d in distances:
    pose = polyline_se2.interpolate(float(d))
    ax.plot(pose.x, pose.y, "g.", markersize=6)
    dx = 0.3 * np.cos(pose.yaw)
    dy = 0.3 * np.sin(pose.yaw)
    ax.annotate(
        "",
        xy=(pose.x + dx, pose.y + dy),
        xytext=(pose.x, pose.y),
        arrowprops=dict(arrowstyle="->", color="green", lw=1, alpha=0.7),
    )

ax.plot([], [], "r->", label="Waypoint heading")
ax.plot([], [], "g->", label="Interpolated heading")

ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_title("PolylineSE2: Path with Heading")
plt.show()

### 4.7.3 `Polyline3D`

In [ ]:
# Create a 3D polyline
points_3d = np.array(
    [
        [0.0, 0.0, 0.0],
        [1.0, 1.0, 1.0],
        [2.0, 0.0, 2.0],
        [3.0, 1.0, 1.5],
    ]
)
polyline_3d = Polyline3D.from_array(points_3d)

print("Length:\t\t", round(polyline_3d.length, 4))

# Interpolate along the 3D path
mid_3d = polyline_3d.interpolate(polyline_3d.length / 2)
print("Midpoint:\t", mid_3d)

# Convert to 2D polyline
polyline_2d = polyline_3d.polyline_2d
print("\n2D projection length:", round(polyline_2d.length, 4))

## 4.8 Coordinate Transforms

The `py123d.geometry.transform` module provides functions to convert poses and points between coordinate frames. This is essential in autonomous driving where data is represented in different frames (e.g., global, ego vehicle, sensor).

Three main operations are available:
- `abs_to_rel`: Convert from absolute (global) to relative (local) coordinates
- `rel_to_abs`: Convert from relative (local) to absolute (global) coordinates
- `reframe`: Convert directly between two reference frames

### 4.8.1 Absolute <-> Relative (SE2)

In [ ]:
from py123d.geometry.transform import (
    abs_to_rel_point_2d,
    abs_to_rel_se2,
    rel_to_abs_point_2d,
    rel_to_abs_se2,
)

# Define an origin (e.g., ego vehicle at position (3, 2) facing 90 degrees)
origin_se2 = PoseSE2(x=3.0, y=2.0, yaw=np.pi / 2)

# A point in absolute coordinates
abs_point_2d = Point2D(x=5.0, y=4.0)
abs_pose_se2 = PoseSE2(x=5.0, y=4.0, yaw=np.pi / 4)
# Transform the point from absolute to relative coordinates (relative to the ego vehicle)
rel_point_2d = abs_to_rel_point_2d(origin_se2, abs_point_2d)
print(f"Absolute point:\t\t{abs_point_2d}")
print(f"Relative to ego:\t{rel_point_2d}")

# Transform back to absolute coordinates to verify
abs_point_recovered = rel_to_abs_point_2d(origin_se2, rel_point_2d)
print(f"Back to absolute:\t{abs_point_recovered}")
print(f"Match:\t\t\t{np.allclose(abs_point_2d.array, abs_point_recovered.array)}")

# Do the same for a PoseSE2
rel_pose = abs_to_rel_se2(origin_se2, abs_pose_se2)
print(f"\nAbsolute pose:\t\t{abs_pose_se2}")
print(f"Relative to ego:\t{rel_pose}")

# Convert back to absolute
abs_pose_recovered = rel_to_abs_se2(origin_se2, rel_pose)
print(f"Back to absolute:\t{abs_pose_recovered}")
print(f"Match:\t\t\t{np.allclose(abs_pose_se2.array, abs_pose_recovered.array)}\n")

In [ ]:
# Visualize coordinate transformation
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Absolute frame
ax = axes[0]
ax.set_title("Absolute (Global) Frame")

# Draw origin (ego vehicle)
ax.plot(origin_se2.x, origin_se2.y, "rs", markersize=12, label="Ego vehicle")
arrow_len = 1.5
ax.annotate(
    "",
    xy=(origin_se2.x + arrow_len * np.cos(origin_se2.yaw), origin_se2.y + arrow_len * np.sin(origin_se2.yaw)),
    xytext=(origin_se2.x, origin_se2.y),
    arrowprops=dict(arrowstyle="->", color="red", lw=2),
)

# Draw some objects in absolute coordinates
objects_abs = [Point2D(5.0, 4.0), Point2D(1.0, 4.0), Point2D(4.0, 0.0)]
for i, obj in enumerate(objects_abs):
    ax.plot(obj.x, obj.y, "bo", markersize=10)
    ax.annotate(f"  Obj {i}", (obj.x, obj.y), fontsize=10)

ax.set_xlim(-1, 7)
ax.set_ylim(-2, 6)
ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
ax.set_xlabel("Global X")
ax.set_ylabel("Global Y")
ax.legend()

# Right: Relative (ego) frame
ax = axes[1]
ax.set_title("Relative (Ego) Frame")

# Ego is at origin in its own frame
ax.plot(0, 0, "rs", markersize=12, label="Ego vehicle")
ax.annotate("", xy=(arrow_len, 0), xytext=(0, 0), arrowprops=dict(arrowstyle="->", color="red", lw=2))

# Transform objects to ego frame
for i, obj in enumerate(objects_abs):
    rel_obj = abs_to_rel_point_2d(origin_se2, obj)
    ax.plot(rel_obj.x, rel_obj.y, "bo", markersize=10)
    ax.annotate(f"  Obj {i}", (rel_obj.x, rel_obj.y), fontsize=10)

ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
ax.set_xlabel("Ego X (forward)")
ax.set_ylabel("Ego Y (left)")
ax.legend()

plt.tight_layout()
plt.show()

### 4.8.2 Absolute <-> Relative (SE3)

In [ ]:
from py123d.geometry.transform import abs_to_rel_point_3d, abs_to_rel_se3, rel_to_abs_point_3d, rel_to_abs_se3

# Define a 3D origin
origin_3d = PoseSE3.from_R_t(
    rotation=EulerAngles(roll=0.0, pitch=0.0, yaw=np.pi / 4),
    translation=Point3D(10.0, 5.0, 1.0),
)

# A 3D point in absolute coordinates
abs_point_3d = Point3D(12.0, 7.0, 2.0)

# Transform to relative coordinates
rel_point_3d = abs_to_rel_point_3d(origin_3d, abs_point_3d)
print(f"Absolute:\t{abs_point_3d}")
print(f"Relative:\t{rel_point_3d}")

# Transform back
recovered = rel_to_abs_point_3d(origin_3d, rel_point_3d)
print(f"Recovered:\t{recovered}")
print(f"Match:\t\t{np.allclose(abs_point_3d.array, recovered.array)}")

# SE3 pose transformation
abs_pose_se2 = PoseSE3.from_R_t(
    rotation=EulerAngles(roll=0.0, pitch=0.0, yaw=np.pi),
    translation=Point3D(15.0, 10.0, 2.0),
)
rel_pose = abs_to_rel_se3(origin_3d, abs_pose_se2)
recovered_pose = rel_to_abs_se3(origin_3d, rel_pose)
print(f"\nPose roundtrip match: {np.allclose(abs_pose_se2.array, recovered_pose.array)}")

### 4.8.3 Reframing Between Frames

`reframe` converts directly between two reference frames without going through absolute coordinates explicitly. This is useful when transforming data from one sensor's frame to another.

In [ ]:
from py123d.geometry.transform import reframe_point_2d, reframe_se2

# Two reference frames (e.g., two different timesteps of the ego vehicle)
frame_t0 = PoseSE2(x=0.0, y=0.0, yaw=0.0)  # t=0: at origin, facing forward
frame_t1 = PoseSE2(x=5.0, y=2.0, yaw=np.pi / 2)  # t=1: moved and rotated

# A point known in frame_t0
point_in_t0 = Point2D(x=3.0, y=1.0)
pose_in_t0 = PoseSE2(x=3.0, y=1.0, yaw=0.0)


# Convert to frame_t1
point_in_t1 = reframe_point_2d(from_origin=frame_t0, to_origin=frame_t1, point_2d=point_in_t0)
pose_in_t1 = reframe_se2(from_origin=frame_t0, to_origin=frame_t1, pose_se2=pose_in_t0)

print(f"Point in t0 frame:\t{point_in_t0}")
print(f"Point in t1 frame:\t{point_in_t1}")
print(f"Pose in t1 frame:\t{pose_in_t1}")

### 4.8.4 Body-Frame Translations

Translate a pose along its own coordinate axes (e.g., move a vehicle forward or sideways):

In [ ]:
from py123d.geometry.transform import translate_se2_along_body_frame, translate_se2_along_x, translate_se2_along_y

# A pose facing 45 degrees
pose = PoseSE2(x=0.0, y=0.0, yaw=np.pi / 4)

# Translate along the body's forward (x) direction
forward = translate_se2_along_x(pose, distance=3.0)
print(f"Original:\t{pose}")
print(f"Forward 3m:\t{forward}")

# Translate along the body's left (y) direction
left = translate_se2_along_y(pose, distance=2.0)
print(f"Left 2m:\t{left}")

# Arbitrary body-frame translation
custom = translate_se2_along_body_frame(pose, Vector2D(3.0, 2.0))
print(f"Custom (3,2):\t{custom}")

In [ ]:
# Visualize body-frame translations
fig, ax = plt.subplots(figsize=(8, 8))


def draw_pose(ax, pose, color, label, arrow_len=1.0):
    ax.plot(pose.x, pose.y, "o", color=color, markersize=10)
    dx = arrow_len * np.cos(pose.yaw)
    dy = arrow_len * np.sin(pose.yaw)
    ax.annotate(
        "",
        xy=(pose.x + dx, pose.y + dy),
        xytext=(pose.x, pose.y),
        arrowprops=dict(arrowstyle="->", color=color, lw=2),
    )
    ax.annotate(f"  {label}", (pose.x, pose.y), fontsize=10, color=color)


draw_pose(ax, pose, "black", "Original")
draw_pose(ax, forward, "blue", "Forward 3m")
draw_pose(ax, left, "green", "Left 2m")
draw_pose(ax, custom, "red", "Custom (3, 2)")

# Draw body axes at the original pose
body_x_end = (2.0 * np.cos(pose.yaw), 2.0 * np.sin(pose.yaw))
body_y_end = (2.0 * np.cos(pose.yaw + np.pi / 2), 2.0 * np.sin(pose.yaw + np.pi / 2))
ax.annotate("", xy=body_x_end, xytext=(0, 0), arrowprops=dict(arrowstyle="->", color="grey", lw=1, linestyle="--"))
ax.annotate("", xy=body_y_end, xytext=(0, 0), arrowprops=dict(arrowstyle="->", color="grey", lw=1, linestyle="--"))
ax.text(body_x_end[0] + 0.1, body_x_end[1] + 0.1, "body x", color="grey", fontsize=9)
ax.text(body_y_end[0] + 0.1, body_y_end[1] + 0.1, "body y", color="grey", fontsize=9)

ax.set_xlim(-3, 5)
ax.set_ylim(-2, 5)
ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
ax.set_title("Body-Frame Translations")
ax.set_xlabel("Global X")
ax.set_ylabel("Global Y")
plt.show()

### 4.8.5 Batch Transforms

Array functions (suffixed with `_array`) operate on batches of poses or points for efficient vectorized computation:

In [ ]:
from py123d.geometry import PoseSE2Index
from py123d.geometry.transform import abs_to_rel_points_2d_array, abs_to_rel_se2_array

# Ego vehicle origin
origin_se2 = PoseSE2(x=5.0, y=5.0, yaw=np.pi / 4)

# Batch of 100 random points in absolute coordinates
np.random.seed(42)
abs_points = np.random.uniform(0, 10, size=(100, 2))

# Batch transform to relative coordinates
rel_points = abs_to_rel_points_2d_array(origin_se2, abs_points)

print(f"Input shape:\t{abs_points.shape}")
print(f"Output shape:\t{rel_points.shape}")

# Batch SE2 transform
abs_poses = np.random.uniform(0, 10, size=(50, 3))  # 50 random poses
abs_poses[:, PoseSE2Index.YAW] = np.random.uniform(-np.pi, np.pi, size=50)
rel_poses = abs_to_rel_se2_array(origin_se2, abs_poses)
print(f"\nSE2 batch: {abs_poses.shape} -> {rel_poses.shape}")

In [ ]:
# Visualize batch transform
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, pts, title, xlabel, ylabel in [
    (axes[0], abs_points, "Absolute Frame", "Global X", "Global Y"),
    (axes[1], rel_points, "Relative (Ego) Frame", "Ego X (forward)", "Ego Y (left)"),
]:
    ax.scatter(pts[:, 0], pts[:, 1], s=15, alpha=0.6, c="steelblue")
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_aspect("equal")
    ax.grid(True, alpha=0.3)

# Draw ego in both frames
for ax, x, y, yaw in [(axes[0], origin_se2.x, origin_se2.y, origin_se2.yaw), (axes[1], 0, 0, 0)]:
    ax.plot(x, y, "rs", markersize=12, label="Ego")
    ax.annotate(
        "",
        xy=(x + 2 * np.cos(yaw), y + 2 * np.sin(yaw)),
        xytext=(x, y),
        arrowprops=dict(arrowstyle="->", color="red", lw=2),
    )
    ax.legend()

plt.tight_layout()
plt.show()

## 4.9 NumPy Interoperability

All geometry objects implement `__array__`, so they work directly with NumPy functions:

In [ ]:
point = Point3D(1.0, 2.0, 3.0)

# Direct conversion to numpy array
arr = np.array(point)
print("np.array(point):\t", arr)
print("dtype:\t\t\t", arr.dtype)

# With dtype casting
arr_f32 = np.array(point, dtype=np.float32)
print("as float32:\t\t", arr_f32)

# Indexing and slicing
print("\npoint[0]:\t\t", point[0])
print("point[:2]:\t\t", point[:2])
print("len(point):\t\t", len(point))
print("point.shape:\t\t", point.shape)
print("point.tolist():\t\t", point.tolist())

# Equality comparison
p1 = Point3D(1.0, 2.0, 3.0)
p2 = Point3D(1.0, 2.0, 3.0)
p3 = Point3D(4.0, 5.0, 6.0)
print("\np1 == p2:\t\t", p1 == p2)
print("p1 == p3:\t\t", p1 == p3)

list_of_points = [Point3D(1.0, 2.0, 3.0), Point3D(4.0, 5.0, 6.0)]
print("\nList of points:\t\t", list_of_points)
array_of_points = np.array(list_of_points)
print("Array of points:\t", array_of_points)

## 4.10 Performance: Think in Arrays, Not Objects

The typed geometry classes (`Point3D`, `PoseSE3`, `Vector3D`, etc.) are designed for **readability, type safety, and single-element access**. They are ideal for writing clear, self-documenting code when working with individual values.

However, when processing **batches of data** (e.g., transforming a Lidar point cloud or a trajectory of poses), you should always operate on raw NumPy arrays using the `_array`-suffixed functions. The performance difference is dramatic because:

1. **No Python-level object creation** -- each `Point3D(...)` or `PoseSE3(...)` involves Python overhead.
2. **Vectorized math** -- NumPy dispatches the entire batch to optimized C/BLAS routines in a single call.
3. **Cache-friendly memory access** -- contiguous arrays are processed efficiently by the CPU.

Let's measure the difference.

In [ ]:
import timeit

from py123d.geometry.transform import abs_to_rel_point_3d, abs_to_rel_points_3d_array

# A reference frame (e.g., ego vehicle pose)
origin = PoseSE3.from_R_t(
    rotation=EulerAngles(roll=0.1, pitch=0.2, yaw=0.3),
    translation=Point3D(10.0, 20.0, 5.0),
)

# Generate a batch of 10,000 random 3D points (e.g., a Lidar point cloud)
np.random.seed(0)
N = 10_000
points_batch = np.random.uniform(-50, 50, size=(N, 3))


# --- Approach 1: Python for-loop with typed objects ---
def transform_loop():
    return np.array([abs_to_rel_point_3d(origin, Point3D(*p)).array for p in points_batch])


# --- Approach 2: Vectorized batch transform ---
def transform_batch():
    return abs_to_rel_points_3d_array(origin, points_batch)


# Verify both produce the same result
result_loop = transform_loop()
result_batch = transform_batch()
print(f"Results match: {np.allclose(result_loop, result_batch)}")

# Benchmark
n_repeats = 5
t_loop = timeit.timeit(transform_loop, number=n_repeats) / n_repeats
t_batch = timeit.timeit(transform_batch, number=n_repeats) / n_repeats

print(f"\nTransforming {N:,} points from absolute to relative coordinates:")
print(f"  For-loop (typed objects): {t_loop * 1000:.2f} ms")
print(f"  Batch (NumPy array):      {t_batch * 1000:.2f} ms")
print(f"  Speedup:                  {t_loop / t_batch:.0f}x")

In [ ]:
# How does the gap scale with batch size?
batch_sizes = [10, 100, 1_000, 10_000, 100_000]
times_loop = []
times_batch = []

for n in batch_sizes:
    pts = np.random.uniform(-50, 50, size=(n, 3))

    # What not to do:
    def _loop(pts=pts):
        return np.array([abs_to_rel_point_3d(origin, Point3D(*p)).array for p in pts])

    # What to do:
    def _batch(pts=pts):
        return abs_to_rel_points_3d_array(origin, pts)

    repeats = max(1, 200_000 // n)  # fewer repeats for large batches
    times_loop.append(timeit.timeit(_loop, number=repeats) / repeats * 1000)
    times_batch.append(timeit.timeit(_batch, number=repeats) / repeats * 1000)

# Plot
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(batch_sizes, times_loop, "o-", label="For-loop (typed objects)", color="tab:red")
ax.loglog(batch_sizes, times_batch, "s-", label="Batch (NumPy array)", color="tab:blue")
ax.set_xlabel("Number of points")
ax.set_ylabel("Time [ms]")
ax.set_title("SE3 Point Transform: For-Loop vs. Batch")
ax.legend()
ax.grid(True, alpha=0.3, which="both")

# Annotate speedup
for n, tl, tb in zip(batch_sizes, times_loop, times_batch):
    ax.annotate(
        f"{tl / tb:.0f}x",
        xy=(n, tb),
        xytext=(0, -18),
        textcoords="offset points",
        fontsize=9,
        ha="center",
        color="tab:blue",
    )

plt.tight_layout()
plt.show()

Currently, 123D uses NumPy for simplicity. Because all batch operations are expressed as array computations (matrix multiplications, broadcasting, einsum), the same logic could be extended to PyTorch or JAX in the future.

You made it to the end. For further information, visit the [documentation](https://autonomousvision.github.io/py123d/) or check out the other tutorials for working with scenes, maps, and visualizations.